In [ ]:
# Use this cell to define your strategy
from itertools import count


def my_strategy(board, player_number):
    '''Write your strategy here
    (param) board ([[int]*6]*7): The board the move will be taken on. For each position, will be 0 for unoccupied, 1 for occupied by player 1, 2 for occupied by player 2.
    (param) int player_number The number of your pieces in the board
    (return) int the number of the column you want your next move to be in (0 for the leftmost column, 6 for the rightmost)'''
    #Write your code here
    import random
    import numpy as np
    
    valid_moves = [c for c in range(7) if board[c][5] == 0]     # Check for valid columns (not full, top row is 0)
    
    if player_number == 1:
        
        if board == np.zeros((6,7)).tolist():   # if it's the first move of the game (empty board)
            return 3    # take center column if available on first move

        # check columns
        for i in range(7):
            op_count = 0        # count of opponent pieces in a column
            own_count = 0       # count of own pieces in a column
            if board[i][5] == 0:    # if the top row is empty, no need to check this column
                for j in range(6):
                    if board[i][j] == 1:
                        op_count = 0       # reset op_count if own piece found
                        own_count = own_count + 1
                    elif board[i][j] == 2:
                        own_count = 0       # reset own_count if opponent piece found
                        op_count = op_count + 2
                    else: 
                        if op_count == 6 or own_count == 3:      # if there are 3 consecutive pieces in the same column
                            return i        # return the column to block opponent's or to win myself
                        else:
                            break       # empty space found, stop checking this column, move to next column
            else:
                continue
            
        # check for rows
        for j in range(6):
            op_count = 0
            own_count = 0
            for i in range(7):
                if board[i][j] == 1:
                    op_count = 0       # reset op_count if own piece found
                    own_count = own_count + 1
                elif board[i][j] == 2:
                    own_count = 0       # reset own_count if opponent piece found
                    op_count = op_count + 2
                else: 
                    if op_count == 6 or own_count == 3:      # if there are 3 consecutive pieces in the same row
                        return i        # return the column to block opponent's or to win myself
                    else:
                        break       # empty space found, stop checking this row, move to next row
             
        
        
        return random.choice(valid_moves)  # Random valid move as a placeholder strategy
    
    else:   # player_number == 2
        
        # check columns
        for i in range(7):
            op_count = 0
            own_count = 0
            if board[i][5] == 0: 
                for j in range(6):
                    if board[i][j] == 2:
                        op_count = 0       # reset count if own piece found
                        own_count = own_count + 1
                    elif board[i][j] == 1:
                        op_count = op_count + 1
                    else: 
                        if op_count == 3 or own_count == 6:      # if there are 3 consecutive pieces in the same column
                            return i        # return the column to block opponent's win
                        else:
                            break       # empty space found, stop checking this column, move to next column
            else:
                continue
        
        # check for rows
        for j in range(6):
            op_count = 0
            own_count = 0
            for i in range(7):
                if board[i][j] == 2:
                    op_count = 0       # reset count if own piece found
                    own_count = own_count + 1
                elif board[i][j] == 1:
                    op_count = op_count + 1
                else: 
                    if op_count == 3 or own_count == 6:      # if there are 3 consecutive pieces in the same row
                        return i        # return the column to block opponent's win
                    else:
                        break       # empty space found, stop checking this row, move to next row                    
        
        
        return random.choice(valid_moves)  # Random valid move as a placeholder strategy
    

In [58]:
# Final strategy
import time
import math
import random

ROWS = 6
COLS = 7
MOVE_ORDER = (3, 2, 4, 1, 5, 0, 6)

# Precompute all 4-cell windows in (col,row) coordinates
WINDOWS = []
# horizontal
for r in range(ROWS):
    for c in range(COLS - 3):
        WINDOWS.append(((c, r), (c+1, r), (c+2, r), (c+3, r)))
# vertical
for c in range(COLS):
    for r in range(ROWS - 3):
        WINDOWS.append(((c, r), (c, r+1), (c, r+2), (c, r+3)))
# diag /
for c in range(COLS - 3):
    for r in range(ROWS - 3):
        WINDOWS.append(((c, r), (c+1, r+1), (c+2, r+2), (c+3, r+3)))
# diag \
for c in range(COLS - 3):
    for r in range(3, ROWS):
        WINDOWS.append(((c, r), (c+1, r-1), (c+2, r-2), (c+3, r-3)))


def my_strategy(board, player_number):
    """
    board[col][row], row=0 bottom, values 0/1/2.
    Must return an int 0..6 for a non-full column.
    """
    start = time.perf_counter()

    # Much faster than before: give yourself a smaller budget.
    # This makes total tournament time MUCH lower.
    TIME_BUDGET = 0.08  # tune: 0.08–0.20 depending on scoring

    def time_up():
        return (time.perf_counter() - start) >= TIME_BUDGET

    # Always compute a legal fallback first (prevents forfeits)
    try:
        heights = compute_heights(board)
        legal = legal_moves(heights)
        fallback = legal[0] if legal else 0
    except Exception:
        return 0

    try:
        me = 1 if int(player_number) == 1 else 2
        opp = 2 if me == 1 else 1

        # If board format is weird, just return something legal
        if not board_shape_ok(board):
            return fallback

        # First move: center
        if sum(heights) == 0:
            return 3 if heights[3] < ROWS else fallback

        legal = legal_moves(heights)
        if not legal:
            return fallback

        # Tactical: win now / block now
        w = immediate_win(board, heights, me)
        if w is not None:
            return w
        b = immediate_win(board, heights, opp)
        if b is not None:
            return b

        # Root anti-blunder: avoid moves that allow opponent immediate win
        candidates = []
        for c in ordered(legal):
            r = make_move(board, heights, c, me)
            lose = (immediate_win(board, heights, opp) is not None)
            undo_move(board, heights, c, r)
            if not lose:
                candidates.append(c)
        if not candidates:
            candidates = ordered(legal)

        # Fixed shallow depth (FAST). 4 is a good starting point.
        # If you want even faster: set to 3.
        MAX_DEPTH = 4

        best_col = candidates[0]
        best_val = -math.inf

        # Alpha-beta root
        alpha = -math.inf
        beta = math.inf

        for c in candidates:
            if time_up():
                break
            r = make_move(board, heights, c, me)
            if win_from_last(board, c, r, me):
                undo_move(board, heights, c, r)
                return c
            val = -negamax(board, heights, MAX_DEPTH - 1, -beta, -alpha,
                           to_move=opp, other=me, root_player=me,
                           time_up=time_up, last_move=(c, r, me))
            undo_move(board, heights, c, r)

            if val > best_val:
                best_val = val
                best_col = c
            if val > alpha:
                alpha = val

        return ensure_valid(best_col, heights)

    except Exception:
        # Never forfeit on exception
        return fallback


# -------------------------
# Fast negamax + alpha-beta (no TT, no hashing)
# -------------------------

def negamax(board, heights, depth, alpha, beta, to_move, other, root_player, time_up, last_move):
    if time_up():
        return evaluate(board, heights, root_player)

    lc, lr, lp = last_move
    if win_from_last(board, lc, lr, lp):
        # last player just won => current player is losing
        return -10_000_000 + (5 - depth)

    if sum(heights) == ROWS * COLS:
        return 0

    if depth == 0:
        return evaluate(board, heights, root_player)

    best = -math.inf
    legal = legal_moves(heights)
    if not legal:
        return 0

    for c in ordered(legal):
        if time_up():
            break
        r = make_move(board, heights, c, to_move)
        if win_from_last(board, c, r, to_move):
            # immediate win for mover => terrible for current player after negamax sign flip,
            # but we handle it normally by returning big value
            val = 10_000_000 - (5 - depth)
        else:
            val = -negamax(board, heights, depth - 1, -beta, -alpha,
                           to_move=other, other=to_move, root_player=root_player,
                           time_up=time_up, last_move=(c, r, to_move))
        undo_move(board, heights, c, r)

        if val > best:
            best = val
        if val > alpha:
            alpha = val
        if alpha >= beta:
            break

    return best


# -------------------------
# Tactics / legality
# -------------------------

def board_shape_ok(board):
    return isinstance(board, list) and len(board) == COLS and all(isinstance(col, list) and len(col) == ROWS for col in board)

def compute_heights(board):
    h = [0] * COLS
    for c in range(COLS):
        col = board[c]
        i = 0
        while i < ROWS and col[i] != 0:
            i += 1
        h[c] = i
    return h

def legal_moves(heights):
    return [c for c in range(COLS) if heights[c] < ROWS]

def ordered(moves):
    s = set(moves)
    return [c for c in MOVE_ORDER if c in s]

def ensure_valid(col, heights):
    try:
        col = int(col)
    except Exception:
        col = 3
    if 0 <= col < COLS and heights[col] < ROWS:
        return col
    m = legal_moves(heights)
    return m[0] if m else 0

def make_move(board, heights, col, player):
    r = heights[col]
    board[col][r] = player
    heights[col] = r + 1
    return r

def undo_move(board, heights, col, row):
    board[col][row] = 0
    heights[col] = row

def immediate_win(board, heights, player):
    for c in ordered(legal_moves(heights)):
        r = make_move(board, heights, c, player)
        won = win_from_last(board, c, r, player)
        undo_move(board, heights, c, r)
        if won:
            return c
    return None


# -------------------------
# Win check around last move (fast)
# -------------------------

def win_from_last(board, c, r, player):
    return (
        count_dir(board, c, r, 1, 0, player) + count_dir(board, c, r, -1, 0, player) >= 3 or
        count_dir(board, c, r, 0, 1, player) + count_dir(board, c, r, 0, -1, player) >= 3 or
        count_dir(board, c, r, 1, 1, player) + count_dir(board, c, r, -1, -1, player) >= 3 or
        count_dir(board, c, r, 1, -1, player) + count_dir(board, c, r, -1, 1, player) >= 3
    )

def count_dir(board, c, r, dc, dr, player):
    n = 0
    cc = c + dc
    rr = r + dr
    while 0 <= cc < COLS and 0 <= rr < ROWS and board[cc][rr] == player:
        n += 1
        cc += dc
        rr += dr
    return n


# -------------------------
# Window heuristic (gravity-aware, but cheap)
# -------------------------

def evaluate(board, heights, me):
    opp = 2 if me == 1 else 1
    score = 0

    # Center control (very cheap)
    score += 6 * sum(1 for r in range(ROWS) if board[3][r] == me)

    for (a, b, c, d) in WINDOWS:
        v0 = board[a[0]][a[1]]
        v1 = board[b[0]][b[1]]
        v2 = board[c[0]][c[1]]
        v3 = board[d[0]][d[1]]

        mc = (v0 == me) + (v1 == me) + (v2 == me) + (v3 == me)
        oc = (v0 == opp) + (v1 == opp) + (v2 == opp) + (v3 == opp)
        if mc and oc:
            continue

        empties = 4 - mc - oc
        if mc == 4:
            score += 100000
            continue
        if oc == 4:
            score -= 100000
            continue

        # playable empties: (col,row) is playable if row == heights[col]
        pe = 0
        if v0 == 0 and a[1] == heights[a[0]]: pe += 1
        if v1 == 0 and b[1] == heights[b[0]]: pe += 1
        if v2 == 0 and c[1] == heights[c[0]]: pe += 1
        if v3 == 0 and d[1] == heights[d[0]]: pe += 1

        if mc == 3 and empties == 1:
            score += 120 if pe == 1 else 30
        elif oc == 3 and empties == 1:
            score -= 140 if pe == 1 else 45
        elif mc == 2 and empties == 2:
            score += 10 + 2 * pe
        elif oc == 2 and empties == 2:
            score -= 12 + 2 * pe

    return score

In [62]:
import time
import math
import random

ROWS = 6
COLS = 7
MOVE_ORDER = (3, 2, 4, 1, 5, 0, 6)

# Precompute all 4-cell windows in (col,row) coordinates
WINDOWS = []
# horizontal
for r in range(ROWS):
    for c in range(COLS - 3):
        WINDOWS.append(((c, r), (c+1, r), (c+2, r), (c+3, r)))
# vertical
for c in range(COLS):
    for r in range(ROWS - 3):
        WINDOWS.append(((c, r), (c, r+1), (c, r+2), (c, r+3)))
# diag /
for c in range(COLS - 3):
    for r in range(ROWS - 3):
        WINDOWS.append(((c, r), (c+1, r+1), (c+2, r+2), (c+3, r+3)))
# diag \
for c in range(COLS - 3):
    for r in range(3, ROWS):
        WINDOWS.append(((c, r), (c+1, r-1), (c+2, r-2), (c+3, r-3)))


def my_strategy_slow(board, player_number):
    """
    board[col][row], row=0 bottom, values 0/1/2.
    Must return an int 0..6 for a non-full column.
    """
    start = time.perf_counter()

    # Much faster than before: give yourself a smaller budget.
    # This makes total tournament time MUCH lower.
    TIME_BUDGET = 0.20  # tune: 0.08–0.20 depending on scoring

    def time_up():
        return (time.perf_counter() - start) >= TIME_BUDGET

    # Always compute a legal fallback first (prevents forfeits)
    try:
        heights = compute_heights(board)
        legal = legal_moves(heights)
        fallback = legal[0] if legal else 0
    except Exception:
        return 0

    try:
        me = 1 if int(player_number) == 1 else 2
        opp = 2 if me == 1 else 1

        # If board format is weird, just return something legal
        if not board_shape_ok(board):
            return fallback

        # First move: center
        if sum(heights) == 0:
            return 3 if heights[3] < ROWS else fallback

        legal = legal_moves(heights)
        if not legal:
            return fallback

        # Tactical: win now / block now
        w = immediate_win(board, heights, me)
        if w is not None:
            return w
        b = immediate_win(board, heights, opp)
        if b is not None:
            return b

        # Root anti-blunder: avoid moves that allow opponent immediate win
        candidates = []
        for c in ordered(legal):
            r = make_move(board, heights, c, me)
            lose = (immediate_win(board, heights, opp) is not None)
            undo_move(board, heights, c, r)
            if not lose:
                candidates.append(c)
        if not candidates:
            candidates = ordered(legal)

        # Fixed shallow depth (FAST). 4 is a good starting point.
        # If you want even faster: set to 3.
        MAX_DEPTH = 4

        best_col = candidates[0]
        best_val = -math.inf

        # Alpha-beta root
        alpha = -math.inf
        beta = math.inf

        for c in candidates:
            if time_up():
                break
            r = make_move(board, heights, c, me)
            if win_from_last(board, c, r, me):
                undo_move(board, heights, c, r)
                return c
            val = -negamax(board, heights, MAX_DEPTH - 1, -beta, -alpha,
                           to_move=opp, other=me, root_player=me,
                           time_up=time_up, last_move=(c, r, me))
            undo_move(board, heights, c, r)

            if val > best_val:
                best_val = val
                best_col = c
            if val > alpha:
                alpha = val

        return ensure_valid(best_col, heights)

    except Exception:
        # Never forfeit on exception
        return fallback


# -------------------------
# Fast negamax + alpha-beta (no TT, no hashing)
# -------------------------

def negamax(board, heights, depth, alpha, beta, to_move, other, root_player, time_up, last_move):
    if time_up():
        return evaluate(board, heights, root_player)

    lc, lr, lp = last_move
    if win_from_last(board, lc, lr, lp):
        # last player just won => current player is losing
        return -10_000_000 + (5 - depth)

    if sum(heights) == ROWS * COLS:
        return 0

    if depth == 0:
        return evaluate(board, heights, root_player)

    best = -math.inf
    legal = legal_moves(heights)
    if not legal:
        return 0

    for c in ordered(legal):
        if time_up():
            break
        r = make_move(board, heights, c, to_move)
        if win_from_last(board, c, r, to_move):
            # immediate win for mover => terrible for current player after negamax sign flip,
            # but we handle it normally by returning big value
            val = 10_000_000 - (5 - depth)
        else:
            val = -negamax(board, heights, depth - 1, -beta, -alpha,
                           to_move=other, other=to_move, root_player=root_player,
                           time_up=time_up, last_move=(c, r, to_move))
        undo_move(board, heights, c, r)

        if val > best:
            best = val
        if val > alpha:
            alpha = val
        if alpha >= beta:
            break

    return best


# -------------------------
# Tactics / legality
# -------------------------

def board_shape_ok(board):
    return isinstance(board, list) and len(board) == COLS and all(isinstance(col, list) and len(col) == ROWS for col in board)

def compute_heights(board):
    h = [0] * COLS
    for c in range(COLS):
        col = board[c]
        i = 0
        while i < ROWS and col[i] != 0:
            i += 1
        h[c] = i
    return h

def legal_moves(heights):
    return [c for c in range(COLS) if heights[c] < ROWS]

def ordered(moves):
    s = set(moves)
    return [c for c in MOVE_ORDER if c in s]

def ensure_valid(col, heights):
    try:
        col = int(col)
    except Exception:
        col = 3
    if 0 <= col < COLS and heights[col] < ROWS:
        return col
    m = legal_moves(heights)
    return m[0] if m else 0

def make_move(board, heights, col, player):
    r = heights[col]
    board[col][r] = player
    heights[col] = r + 1
    return r

def undo_move(board, heights, col, row):
    board[col][row] = 0
    heights[col] = row

def immediate_win(board, heights, player):
    for c in ordered(legal_moves(heights)):
        r = make_move(board, heights, c, player)
        won = win_from_last(board, c, r, player)
        undo_move(board, heights, c, r)
        if won:
            return c
    return None


# -------------------------
# Win check around last move (fast)
# -------------------------

def win_from_last(board, c, r, player):
    return (
        count_dir(board, c, r, 1, 0, player) + count_dir(board, c, r, -1, 0, player) >= 3 or
        count_dir(board, c, r, 0, 1, player) + count_dir(board, c, r, 0, -1, player) >= 3 or
        count_dir(board, c, r, 1, 1, player) + count_dir(board, c, r, -1, -1, player) >= 3 or
        count_dir(board, c, r, 1, -1, player) + count_dir(board, c, r, -1, 1, player) >= 3
    )

def count_dir(board, c, r, dc, dr, player):
    n = 0
    cc = c + dc
    rr = r + dr
    while 0 <= cc < COLS and 0 <= rr < ROWS and board[cc][rr] == player:
        n += 1
        cc += dc
        rr += dr
    return n


# -------------------------
# Window heuristic (gravity-aware, but cheap)
# -------------------------

def evaluate(board, heights, me):
    opp = 2 if me == 1 else 1
    score = 0

    # Center control (very cheap)
    score += 6 * sum(1 for r in range(ROWS) if board[3][r] == me)

    for (a, b, c, d) in WINDOWS:
        v0 = board[a[0]][a[1]]
        v1 = board[b[0]][b[1]]
        v2 = board[c[0]][c[1]]
        v3 = board[d[0]][d[1]]

        mc = (v0 == me) + (v1 == me) + (v2 == me) + (v3 == me)
        oc = (v0 == opp) + (v1 == opp) + (v2 == opp) + (v3 == opp)
        if mc and oc:
            continue

        empties = 4 - mc - oc
        if mc == 4:
            score += 100000
            continue
        if oc == 4:
            score -= 100000
            continue

        # playable empties: (col,row) is playable if row == heights[col]
        pe = 0
        if v0 == 0 and a[1] == heights[a[0]]: pe += 1
        if v1 == 0 and b[1] == heights[b[0]]: pe += 1
        if v2 == 0 and c[1] == heights[c[0]]: pe += 1
        if v3 == 0 and d[1] == heights[d[0]]: pe += 1

        if mc == 3 and empties == 1:
            score += 120 if pe == 1 else 30
        elif oc == 3 and empties == 1:
            score -= 140 if pe == 1 else 45
        elif mc == 2 and empties == 2:
            score += 10 + 2 * pe
        elif oc == 2 and empties == 2:
            score -= 12 + 2 * pe

    return score

In [78]:
import time
import math

ROWS, COLS = 6, 7
ORDER = (3, 2, 4, 1, 5, 0, 6)  # center-first

# Precompute all 4-cell windows as (col,row) coordinates
WINDOWS = []
# horizontal
for r in range(ROWS):
    for c in range(COLS - 3):
        WINDOWS.append(((c, r), (c+1, r), (c+2, r), (c+3, r)))
# vertical
for c in range(COLS):
    for r in range(ROWS - 3):
        WINDOWS.append(((c, r), (c, r+1), (c, r+2), (c, r+3)))
# diag /
for c in range(COLS - 3):
    for r in range(ROWS - 3):
        WINDOWS.append(((c, r), (c+1, r+1), (c+2, r+2), (c+3, r+3)))
# diag \
for c in range(COLS - 3):
    for r in range(3, ROWS):
        WINDOWS.append(((c, r), (c+1, r-1), (c+2, r-2), (c+3, r-3)))


def my_strategy_simple(board, player_number):
    """
    board[col][row], row=0 bottom. values: 0 empty, 1/2 players.
    Must return an int column 0..6 that is not full.
    """
    start = time.perf_counter()
    TIME_BUDGET = 0.20  # keeps tournament runtime low; raise slightly if allowed

    def time_up():
        return (time.perf_counter() - start) >= TIME_BUDGET

    # Always compute a legal fallback first (prevents forfeits)
    try:
        heights = compute_heights(board)
        legal = legal_moves(heights)
        fallback = legal[0] if legal else 0
    except Exception:
        return 0

    try:
        me = 1 if int(player_number) == 1 else 2
        opp = 2 if me == 1 else 1

        if not legal:
            return fallback

        # First move: center
        if sum(heights) == 0:
            return 3 if heights[3] < ROWS else fallback

        # Immediate win / immediate block (very strong and cheap)
        w = immediate_win(board, heights, me)
        if w is not None:
            return w
        b = immediate_win(board, heights, opp)
        if b is not None:
            return b

        # Root "anti-blunder": avoid moves that allow opponent immediate win
        cand = []
        for c in ordered(legal):
            r = make(board, heights, c, me)
            bad = (immediate_win(board, heights, opp) is not None)
            undo(board, heights, c, r)
            if not bad:
                cand.append(c)
        if not cand:
            cand = ordered(legal)

        # Fixed depth alpha-beta (simple + fast)
        DEPTH = 5  # set to 3 for ultra-fast, 5 for stronger (if time allows)

        best_col = cand[0]
        best_val = -math.inf
        alpha = -math.inf
        beta = math.inf

        for c in cand:
            if time_up():
                break
            r = make(board, heights, c, me)
            if win_from_last(board, c, r, me):
                undo(board, heights, c, r)
                return c
            val = -negamax(board, heights, DEPTH - 1, -beta, -alpha, opp, me, me, time_up, (c, r, me))
            undo(board, heights, c, r)

            if val > best_val:
                best_val = val
                best_col = c
            if val > alpha:
                alpha = val

        return ensure_valid(best_col, heights)

    except Exception:
        return fallback


# -------------------- Mechanics --------------------

def compute_heights(board):
    """heights[c] = first empty row index in column c."""
    h = [0] * COLS
    for c in range(COLS):
        i = 0
        col = board[c]
        while i < ROWS and col[i] != 0:
            i += 1
        h[c] = i
    return h

def legal_moves(heights):
    return [c for c in range(COLS) if heights[c] < ROWS]

def ordered(moves):
    s = set(moves)
    return [c for c in ORDER if c in s]

def ensure_valid(col, heights):
    try:
        col = int(col)
    except Exception:
        col = 3
    if 0 <= col < COLS and heights[col] < ROWS:
        return col
    m = legal_moves(heights)
    return m[0] if m else 0

def make(board, heights, col, player):
    r = heights[col]
    board[col][r] = player
    heights[col] = r + 1
    return r

def undo(board, heights, col, row):
    board[col][row] = 0
    heights[col] = row


# -------------------- Tactics: win checks --------------------

def immediate_win(board, heights, player):
    for c in ordered(legal_moves(heights)):
        r = make(board, heights, c, player)
        won = win_from_last(board, c, r, player)
        undo(board, heights, c, r)
        if won:
            return c
    return None

def win_from_last(board, c, r, player):
    # 4 directions: horiz, vert, diag, anti-diag
    return (
        count_dir(board, c, r, 1, 0, player) + count_dir(board, c, r, -1, 0, player) >= 3 or
        count_dir(board, c, r, 0, 1, player) + count_dir(board, c, r, 0, -1, player) >= 3 or
        count_dir(board, c, r, 1, 1, player) + count_dir(board, c, r, -1, -1, player) >= 3 or
        count_dir(board, c, r, 1, -1, player) + count_dir(board, c, r, -1, 1, player) >= 3
    )

def count_dir(board, c, r, dc, dr, player):
    n = 0
    c += dc
    r += dr
    while 0 <= c < COLS and 0 <= r < ROWS and board[c][r] == player:
        n += 1
        c += dc
        r += dr
    return n


# -------------------- Negamax + alpha-beta --------------------

def negamax(board, heights, depth, alpha, beta, to_move, other, root, time_up, last_move):
    if time_up():
        return evaluate(board, heights, root)

    lc, lr, lp = last_move
    if win_from_last(board, lc, lr, lp):
        # last player just made a connect-4 => current player is losing
        return -10_000_000 + (4 - depth)

    if sum(heights) == ROWS * COLS:
        return 0

    if depth == 0:
        return evaluate(board, heights, root)

    best = -math.inf
    moves = ordered(legal_moves(heights))
    for c in moves:
        if time_up():
            break
        r = make(board, heights, c, to_move)
        if win_from_last(board, c, r, to_move):
            val = 10_000_000 - (4 - depth)
        else:
            val = -negamax(board, heights, depth - 1, -beta, -alpha, other, to_move, root, time_up, (c, r, to_move))
        undo(board, heights, c, r)

        if val > best:
            best = val
        if val > alpha:
            alpha = val
        if alpha >= beta:
            break

    return best


# -------------------- Heuristic: window scoring (gravity-aware) --------------------

def evaluate(board, heights, me):
    opp = 2 if me == 1 else 1
    score = 0

    # Center control: simple, effective
    score += 6 * sum(1 for r in range(ROWS) if board[3][r] == me)

    for a, b, c, d in WINDOWS:
        v0 = board[a[0]][a[1]]
        v1 = board[b[0]][b[1]]
        v2 = board[c[0]][c[1]]
        v3 = board[d[0]][d[1]]

        mc = (v0 == me) + (v1 == me) + (v2 == me) + (v3 == me)  # my count
        oc = (v0 == opp) + (v1 == opp) + (v2 == opp) + (v3 == opp)  # opponent count
        # if True = 1, if False = 0
        
        if mc and oc:
            continue

        empties = 4 - mc - oc   
        if mc == 4:
            score += 100000 # my win
            continue
        if oc == 4:
            score -= 100000 # opponent win  
            continue

        # "Playability" under gravity: an empty (col,row) is playable if row == heights[col]
        pe = 0
        if v0 == 0 and a[1] == heights[a[0]]: pe += 1
        if v1 == 0 and b[1] == heights[b[0]]: pe += 1
        if v2 == 0 and c[1] == heights[c[0]]: pe += 1
        if v3 == 0 and d[1] == heights[d[0]]: pe += 1

        # Simple weights (keep small + readable)
        if mc == 3 and empties == 1:
            score += 120 if pe == 1 else 30
        elif oc == 3 and empties == 1:
            score -= 140 if pe == 1 else 45
        elif mc == 2 and empties == 2:
            score += 10 + 2 * pe
        elif oc == 2 and empties == 2:
            score -= 12 + 2 * pe

    return score

In [46]:
# When you want to play against your strategy, run the cell above, then this cell
from game.run_game import run_game
from strategies import manual

run_game("Your Strategy", "You", my_strategy, manual.manual, move_duration=0.1)

You will go first
|               |
|               |
|               |
|               |
|               |
|               |
-----------------
Player You selects column 0
1 Your Strategy
2 You
|               |
|               |
|               |
|               |
|               |
| 2             |
-----------------
Player Your Strategy selects column 3
1 Your Strategy
2 You
|               |
|               |
|               |
|               |
|               |
| 2     1       |
-----------------
Player You selects column 0
1 Your Strategy
2 You
|               |
|               |
|               |
|               |
| 2             |
| 2     1       |
-----------------
Player Your Strategy selects column 0
1 Your Strategy
2 You
|               |
|               |
|               |
| 1             |
| 2             |
| 2     1       |
-----------------
Player You selects column 2
1 Your Strategy
2 You
|               |
|               |
|               |
| 1             |
| 2       

(1, False)

In [ ]:
# Run this cell to see your code play a single match against the opponent you will face in the assessment
# Press enter to progress the game
from game.run_game import run_game
from assessment.assessor import random_opponent

run_game("Your Strategy", "Opponent", my_strategy, my_strategy_slow, move_duration=-1)

Opponent will go first
|               |
|               |
|               |
|               |
|               |
|               |
-----------------
Player Opponent selects column 3
1 Your Strategy
2 Opponent
|               |
|               |
|               |
|               |
|               |
|       2       |
-----------------
Player Your Strategy selects column 1
1 Your Strategy
2 Opponent
|               |
|               |
|               |
|               |
|               |
|   1   2       |
-----------------
Player Opponent selects column 4
1 Your Strategy
2 Opponent
|               |
|               |
|               |
|               |
|               |
|   1   2 2     |
-----------------
Player Your Strategy selects column 5
1 Your Strategy
2 Opponent
|               |
|               |
|               |
|               |
|               |
|   1   2 2 1   |
-----------------
Player Opponent selects column 3
1 Your Strategy
2 Opponent
|               |
|               |
|

(-1, False)

In [79]:
# When you're ready to run your strategy run the top cell, then this cell
# You can do this as often as you like as you improve your strategy
from assessment.assessor import assess

assess(my_strategy_simple)

                                                                                                    
Results
Wins: 991
Draws: 0
Losses: 9
Forfeits: 0
Mark: 97.99%
